In [0]:
%pip install dbldatagen
%pip install Faker

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from dbldatagen import DataGenerator , PyfuncText , DateRange
from faker import Faker
from datetime import timedelta, datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, isnan, col, lit, countDistinct,substring,split
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from pyspark.sql.functions import concat_ws
import random as random

In [0]:
#schema type
schema=StructType([
    StructField('_id',StringType(),False),
    StructField('customer_id',StringType(),False),
    StructField('line_item_id',StringType(),False),
    StructField('order_item_id',StringType(),False),
    StructField('shopify_user_id',StringType(),False),
    StructField('api_client_id',StringType(),False),
    StructField('balance',StringType(),False),
    StructField('created_at',TimestampType(),False), #
    StructField('currency',StringType(),False),
    StructField('disabled_at',StringType(),False), #
    StructField('expires_on',StringType(),False), #
    StructField('intial_value',StringType(),False),
    StructField('updated_at',TimestampType(),False), #
])

#lambda functions
fake=Faker()
customer_id=(lambda context,v:str(random.randint(100,1000)))
line_item_id=(lambda context,v:str(random.randint(100,1000)+10))
order_item_id=(lambda context,v:str(random.randint(100,1000)+20))
shopify_user_id=(lambda context,v:str(random.randint(100,1000)+30))
api_client_id=(lambda context,v:str(random.randint(100,1000)+40))
balance=(lambda context,v:str(random.randint(1000,10000)))
currency=(lambda context,v: str(random.choice(['$','₹'])) )
intial_value=(lambda context,v: str(random.randint(1000,10000)))


#data fill
data=(DataGenerator(spark,rows=10,partitions=2)
      .withSchema(schema)
      .withColumnSpec('_id', min=1,max=1000,step=1)
      .withColumnSpec('customer_id',text=PyfuncText(customer_id))
      .withColumnSpec('line_item_id',text=PyfuncText(line_item_id))
      .withColumnSpec('order_item_id',text=PyfuncText(order_item_id))
      .withColumnSpec('shopify_user_id',text=PyfuncText(shopify_user_id))
      .withColumnSpec('api_client_id',text=PyfuncText(api_client_id))
      .withColumnSpec('balance',text=PyfuncText(balance))
      .withColumnSpec('created_at',"date",
        data_range=DateRange("2017-10-01 00:00:00", "2018-10-06 11:55:00", "days=3"),
        random=True)
      .withColumnSpec('currency',text=PyfuncText(currency))
      .withColumnSpec('disabled_at',"date",
        expr="date_add(created_at, cast(floor(rand() * 100 + 1) as int))",
        baseColumn="created_at")
      .withColumnSpec('expires_on',"date",
        expr="date_add(disabled_at, cast(floor(rand() * 100 + 1) as int))",
        baseColumn="disabled_at")
      .withColumnSpec('intial_value',text=PyfuncText(balance))
      .withColumnSpec('updated_at',"date",
        data_range=DateRange("2017-10-01 00:00:00", "2018-10-06 11:55:00", "days=3"),
        random=True)
)


databuild=data.build()
display(databuild)



_id,customer_id,line_item_id,order_item_id,shopify_user_id,api_client_id,balance,created_at,currency,disabled_at,expires_on,intial_value,updated_at
1,864,202,267,895,774,7099,2017-11-27T00:00:00.000+0000,$,2018-02-17,2018-05-22,8025,2018-01-02T00:00:00.000+0000
2,786,885,409,714,850,2399,2017-12-18T00:00:00.000+0000,$,2018-03-13,2018-03-15,5536,2018-01-02T00:00:00.000+0000
3,963,431,979,902,683,4884,2018-08-06T00:00:00.000+0000,₹,2018-11-03,2018-12-23,7329,2017-12-09T00:00:00.000+0000
4,390,478,805,460,350,6549,2018-07-22T00:00:00.000+0000,$,2018-09-18,2018-12-01,1791,2017-12-18T00:00:00.000+0000
5,392,690,966,647,431,5911,2017-10-04T00:00:00.000+0000,₹,2017-12-21,2018-03-27,9606,2018-09-08T00:00:00.000+0000
6,803,821,335,654,847,8981,2018-06-16T00:00:00.000+0000,$,2018-08-16,2018-10-04,9638,2018-02-13T00:00:00.000+0000
7,294,660,720,773,1012,6778,2018-04-08T00:00:00.000+0000,$,2018-04-16,2018-07-16,3462,2018-04-26T00:00:00.000+0000
8,623,909,838,427,834,6416,2018-06-19T00:00:00.000+0000,$,2018-09-05,2018-12-04,8794,2018-08-27T00:00:00.000+0000
9,781,939,527,814,859,5944,2017-12-06T00:00:00.000+0000,₹,2018-02-11,2018-02-14,3084,2017-10-13T00:00:00.000+0000
10,394,259,122,922,307,2479,2017-11-21T00:00:00.000+0000,$,2017-12-04,2018-01-27,3711,2018-05-02T00:00:00.000+0000
